#### 1. 业务场景介绍
咱们先来看看业务场景，假设你现在有一个电商系统，里面有一个支付订单的场景。   
那对一个订单支付之后，我们需要做下面的步骤：
* 更改订单的状态为“已支付”
* 扣减商品库存
* 给会员增加积分
* 创建销售出库单通知仓库发货  

这是一系列比较真实的步骤，无论大家有没有做过电商系统，应该都能理解。

#### 2. 分布式事务
好，业务场景有了，现在我们要更进一步，实现一个 TCC 分布式事务的效果。   
什么意思呢？也就是说: 
* 订单服务-修改订单状态
* 库存服务-扣减库存
* 积分服务-增加积分
* 仓储服务-创建销售出库单。

上述这几个步骤，要么一起成功，要么一起失败，必须是一个整体性的事务。  
举个例子，现在订单的状态都修改为“已支付”了，结果库存服务扣减库存失败。那个商品的库存原来是 100 件，现在卖掉了 2 件，本来应该是 98 件了。  
结果呢？由于库存服务操作数据库异常，导致库存数量还是 100。这不是在坑人么，当然不能允许这种情况发生了！  
但是如果你不用 TCC 分布式事务方案的话，就用个 Spring Cloud 开发这么一个微服务系统，很有可能会干出这种事儿来。  

#### 3. TCC分布式服务的三种接口: Try-Commited-Cancel  
那么现在到底要如何来实现一个 TCC 分布式事务，使得各个服务，要么一起成功？要么一起失败呢？  
首先，订单服务那儿，它的代码大致来说应该是这样子的: 
```java
public class OrderService {

    // 库存服务
    @Autowired
    private InventoryService inventoryService;

    // 积分服务
    @Autowired
    private CreditService creditService;

    // 仓储服务
    @Autowired
    private WmsService wmsService;

    // 对这个订单完成支付
    public void pay(){
        //对本地的的订单数据库修改订单状态为"已支付"
        orderDAO.updateStatus(OrderStatus.PAYED);

        //调用库存服务扣减库存
        inventoryService.reduceStock();

        //调用积分服务增加积分
        creditService.addCredit();

        //调用仓储服务通知发货
        wmsService.saleDelivery();
    }
}
```
* 微服务要提供3种接口供调用
    * **try: 负责接收保存要修改的资源, 做第一步记录**     
      如果你之前看过 Spring Cloud 架构原理那篇文章，同时对 Spring Cloud 有一定的了解的话，应该是可以理解上面那段代码的。   
      其实就是订单服务完成本地数据库操作之后，通过 Spring Cloud 的 Feign 来调用其他的各个服务罢了。   
      但是光是凭借这段代码，是不足以实现 TCC 分布式事务的啊？！兄弟们，别着急，我们对这个订单服务修改点儿代码好不好。    
        * 首先，上面那个订单服务先把自己的状态修改为：OrderStatus.UPDATING。    
           这是啥意思呢？也就是说，在 pay() 那个方法里，你别直接把订单状态修改为已支付啊！你先把订单状态修改为 UPDATING，也就是修改中的意思。这个状态是个没有任何含义的这么一个状态，代表有人正在修改这个状态罢了。    
        * 然后呢，库存服务直接提供的那个 reduceStock() 接口里，也别直接扣减库存啊，你可以是冻结掉库存。   
          举个例子，本来你的库存数量是 100，你别直接 100 - 2 = 98，扣减这个库存！你可以把可销售的库存：100 - 2 = 98，设置为 98 没问题，然后在一个单独的冻结库存的字段里，设置一个 2。也就是说，有 2 个库存是给冻结了。积分服务的 addCredit() 接口也是同理，别直接给用户增加会员积分。你可以先在积分表里的一个预增加积分字段加入积分。比如：用户积分原本是 1190，现在要增加 10 个积分，别直接 1190 + 10 = 1200 个积分啊！你可以保持积分为 1190 不变，在一个预增加字段里，比如说 prepare_add_credit 字段，设置一个 10，表示有 10 个积分准备增加。
        * 仓储服务的 saleDelivery() 接口也是同理啊，你可以先创建一个销售出库单，但是这个销售出库单的状态是“UNKNOWN”。  
          也就是说，刚刚创建这个销售出库单，此时还不确定它的状态是什么呢！
    * **Confirm: 然后就分成两种情况了，第一种情况是比较理想的，那就是各个服务执行自己的那个 Try 操作，都执行成功了**  
    这个时候，就需要依靠 TCC 分布式事务框架来推动后续的执行了。这里简单提一句，如果你要玩儿 TCC 分布式事务，必须引入一款 TCC 分布式事务框架，比如国内开源的 ByteTCC、Himly、TCC-transaction。  
    否则的话，感知各个阶段的执行情况以及推进执行下一个阶段的这些事情，不太可能自己手写实现，太复杂了。  
    如果你在各个服务里引入了一个 TCC 分布式事务的框架，订单服务里内嵌的那个 TCC 分布式事务框架可以感知到，各个服务的 Try 操作都成功了。  
    此时，TCC 分布式事务框架会控制进入 TCC 下一个阶段，第一个 C 阶段，也就是 Confirm 阶段。   
    这个阶段就是将try中尝试的操作彻底入库执行完毕, 消除中间状态  
        * 订单服务里，你可以加入一个 Confirm 的逻辑，就是正式把订单的状态设置为“已支付”
        * 库存服务也是类似的，你可以有一个 InventoryServiceConfirm 类，里面提供一个 reduceStock() 接口的 Confirm 逻辑，这里就是将之前冻结库存字段的 2 个库存扣掉变为 0。这样的话，可销售库存之前就已经变为 98 了，现在冻结的 2 个库存也没了，那就正式完成了库存的扣减。
        * 积分服务也是类似的，可以在积分服务里提供一个 CreditServiceConfirm 类，里面有一个 addCredit() 接口的 Confirm 逻辑，就是将预增加字段的 10 个积分扣掉，然后加入实际的会员积分字段中，从 1190 变为 1120。
        * 仓储服务也是类似，可以在仓储服务中提供一个 WmsServiceConfirm 类，提供一个 saleDelivery() 接口的 Confirm 逻辑，将销售出库单的状态正式修改为“已创建”，可以供仓储管理人员查看和使用，而不是停留在之前的中间状态“UNKNOWN”了。     
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;好了，上面各种服务的 Confirm 的逻辑都实现好了，一旦订单服务里面的 TCC 分布式事务框架感知到各个服务的 Try 阶段都成功了以后，就会执行各个服务的 Confirm 逻辑。
   订单服务内的 TCC 事务框架会负责跟其他各个服务内的 TCC 事务框架进行通信，依次调用各个服务的 Confirm 逻辑。然后，正式完成各个服务的所有业务逻辑的执行。
    * **Cancel: Try阶段有调用失败的微服务**    
    TCC 分布式事务框架只要感知到了任何一个服务的 Try 逻辑失败了，就会跟各个服务内的 TCC 分布式事务框架进行通信，然后调用各个服务的 Cancel 逻辑。  
    举个例子,在 Try 阶段，比如积分服务吧，它执行出错了，此时会怎么样？    
    那订单服务内的 TCC 事务框架是可以感知到的，然后它会决定对整个 TCC 分布式事务进行回滚。  
    也就是说，会执行各个服务的第二个 C 阶段，Cancel 阶段。同样，为了实现这个 Cancel 阶段，各个服务还得加一些代码。   
        * 首先订单服务，它得提供一个 OrderServiceCancel 的类，在里面有一个 pay() 接口的 Cancel 逻辑，就是可以将订单的状态设置为“CANCELED”，也就是这个订单的状态是已取消。
        * 库存服务也是同理，可以提供 reduceStock() 的 Cancel 逻辑，就是将冻结库存扣减掉 2，加回到可销售库存里去，98 + 2 = 100。
        * 积分服务也需要提供 addCredit() 接口的 Cancel 逻辑，将预增加积分字段的 10 个积分扣减掉。
        * 仓储服务也需要提供一个 saleDelivery() 接口的 Cancel 逻辑，将销售出库单的状态修改为“CANCELED”设置为已取消。